In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('../data/ed_output.parquet')

In [3]:
df.head()

,Title,Source,Author,Published Date,URL,Country,Content,ed_results
6458,China’s economic data breakdowns show signs of...,Channel NewsAsia,South China Morning Post,2023-09-15T02:27:48Z,https://www.channelnewsasia.com/asia/china-eco...,Singapore,Apart from a divergence among different income...,"[{'events': [{'offset': array([207, 211]), 'tr..."
3333,5 years’ jail for ‘key cog’ in cryptocurrency ...,Straits Times,Shaffiq Alkhatib,2024-08-06T07:30:00Z,https://www.straitstimes.com/singapore/courts-...,Singapore,SINGAPORE – He was the chief technological off...,"[{'events': [{'offset': array([219, 226]), 'tr..."
5729,"Australia, India at risk of becoming dumping g...",Straits Times,None,2023-11-17T07:18:00Z,https://www.straitstimes.com/world/europe/aust...,Singapore,LONDON – Nations slow to adopt electric vehicl...,"[{'events': [{'offset': array([111, 118]), 'tr..."
5963,China's new home prices rise for second month ...,Straits Times,None,2023-11-01T04:45:00Z,https://www.straitstimes.com/business/chinas-n...,Singapore,BEIJING - China’s new home prices rose for a s...,"[{'events': [{'offset': array([ 9, 14]), 'trig..."
1731,"China to retaliate if Trump boosts tariff, say...",Straits Times,None,2024-11-16T00:24:00Z,https://www.straitstimes.com/asia/east-asia/ch...,Singapore,BEIJING - China will fight back if US Presiden...,"[{'events': [{'offset': array([176, 182]), 'tr..."


In [4]:
pd.set_option("max.colwidth",0)

In [5]:
df.columns

Index(['Title', 'Source', 'Author', 'Published Date', 'URL', 'Country',
       'Content', 'ed_results'],
      dtype='object')

In [15]:
content_df = df[["Title","Content"]]

## Extract the multiple facts and events

In [25]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

# Replace with your OpenAI API key
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain

# Create a ChatOpenAI LLM instance (chat-based model)
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Define the prompt template for event extraction with relationships
prompt_template = PromptTemplate(
    input_variables=["title", "content"],
    template=(
        "Based on the following title and content, extract multiple independent events or facts in JSON format. "
        "Ensure that all fields explicitly mention entities instead of using pronouns. "
        "Include relationships between events using these predefined types:\n"
        "- `TRIGGERED`: Event A directly caused Event B.\n"
        "- `INFLUENCED`: Event A indirectly impacted Event B.\n"
        "- `CONSEQUENCE_OF`: Event A occurred as a direct consequence of Event B.\n"
        "- `RESPONSE_TO`: Event A was a reaction to Event B.\n"
        "- `ESCALATED_FROM`: Event A worsened an existing situation caused by Event B.\n"
        "- `RELATED_TO`: Event A is contextually related to Event B without direct causation.\n"
        "\n"
        "Use the following JSON structure:\n\n"
        "[\n"
        "{{\n"
        '  "title": "{title}",\n'
        '  "event_id": 1,\n'
        '  "main_event": "[Describe the main event concisely]",\n'
        '  "who": "[Explicit mention of persons or organizations involved]",\n'
        '  "what": "[Specific action or incident that occurred]",\n'
        '  "where": "[Explicit location where the event happened]",\n'
        '  "when": "[Timeframe of the event]",\n'
        '  "cause": "[Explicitly describe what led to this event, avoiding pronouns and vague references]",\n'
        '  "consequences": "[Explicitly describe the outcomes or impact, avoiding pronouns and vague references]",\n'
        '  "event_dependencies": [\n'
        '    {{"event_id": 2, "relation": "TRIGGERED"}}\n'
        '  ]\n'
        "}},\n"
        "{{\n"
        '  "event_id": 2,\n'
        '  "main_event": "[Describe another independent event concisely]",\n'
        '  "who": "[Explicit mention of persons or organizations involved]",\n'
        '  "what": "[Specific action or incident that occurred]",\n'
        '  "where": "[Explicit location where the event happened]",\n'
        '  "when": "[Timeframe of the event]",\n'
        '  "cause": "[Explicitly describe what led to this event, avoiding pronouns and vague references]",\n'
        '  "consequences": "[Explicitly describe the outcomes or impact, avoiding pronouns and vague references]",\n'
        '  "event_dependencies": [\n'
        '    {{"event_id": 3, "relation": "INFLUENCED"}}\n'
        '  ]\n'
        "}},\n"
        "{{\n"
        '  "event_id": 3,\n'
        '  "main_event": "[Describe another independent event concisely]",\n'
        '  "who": "[Explicit mention of persons or organizations involved]",\n'
        '  "what": "[Specific action or incident that occurred]",\n'
        '  "where": "[Explicit location where the event happened]",\n'
        '  "when": "[Timeframe of the event]",\n'
        '  "cause": "[Explicitly describe what led to this event, avoiding pronouns and vague references]",\n'
        '  "consequences": "[Explicitly describe the outcomes or impact, avoiding pronouns and vague references]",\n'
        '  "event_dependencies": []\n'
        "}}\n"
        "... additional events if applicable ...\n"
        "]\n\n"
        "Ignore less informative details and focus on extracting distinct, independent events only. "
        "Ensure all extracted facts are suitable for knowledge graph representation.\n\n"
        "Title:\n{title}\n\nContent:\n{content}"
    )
)


# Create a ChatOpenAI LLM instance (chat-based model)
llm = ChatOpenAI(model="gpt-4", temperature=0)


In [19]:
content_df["content_idx"] = [f"idx_{idx}" for idx in range(0,len(content_df))]

/tmp/ipykernel_267719/2136582530.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df["content_idx"] = [f"idx_{idx}" for idx in range(0,len(content_df))]


In [20]:
content_df = content_df[["content_idx","Title","Content"]]

In [23]:
content_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 6458 to 1199
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   content_idx  50 non-null     object
 1   Title        50 non-null     object
 2   Content      50 non-null     object
dtypes: object(3)
memory usage: 1.6+ KB


In [26]:
event_list = []

for row in content_df.iterrows():
    row = row[1]
    title = row["Title"]
    content = row["Content"]
    formatted_prompt = prompt_template.format(title=title, content=content)
    chain = LLMChain(llm=llm, prompt=prompt_template)
    result = chain.run({"title": title, "content": content})
    event_list.append(result)

/tmp/ipykernel_267719/406733826.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)
/tmp/ipykernel_267719/406733826.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run({"title": title, "content": content})


In [27]:
len(event_list)

50

In [31]:
import ast

In [61]:
event_lst = [ast.literal_eval(event) for event in event_list]

In [65]:
event_lst[:1]

[[{'title': 'China’s economic data breakdowns show signs of life amid prevailing doom-and-gloom outlook',
   'event_id': 1,
   'main_event': "China's consumption trend polarised among sectors",
   'who': 'China',
   'what': 'Consumption trend polarised among sectors, with service consumption such as concerts, tourism and catering performing better than the sale of goods',
   'where': 'China',
   'when': 'This year',
   'cause': 'Worsening income prospects suppressing the sale of goods',
   'consequences': 'Service sectors like concerts, tourism and catering have seen better performance',
   'event_dependencies': []},
  {'event_id': 2,
   'main_event': 'Record high booking value on Chinese travel platforms',
   'who': 'Trip.com and Tongcheng Travel',
   'what': 'Record high total booking value',
   'where': 'China',
   'when': 'July',
   'cause': 'High demand during the summer holiday',
   'consequences': 'Increased business for Chinese travel platforms Trip.com and Tongcheng Travel',
 

In [66]:
# Save as JSON file
with open("../data/events_data.json", "w", encoding="utf-8") as json_file:
    json.dump(event_lst, json_file, indent=4, ensure_ascii=False)

print("JSON file saved successfully!")

JSON file saved successfully!


In [67]:
data = pd.read_csv("../data/data.csv")

In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7674 entries, 0 to 7673
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Title           7674 non-null   object
 1   Source          7674 non-null   object
 2   Author          1927 non-null   object
 3   Published Date  7674 non-null   object
 4   URL             7674 non-null   object
 5   Country         7674 non-null   object
 6   Content         7674 non-null   object
dtypes: object(7)
memory usage: 419.8+ KB


In [55]:
from itertools import count

def assign_unique_event_ids(data):
    # Flatten the list of lists
    all_events = [event for sublist in data for event in sublist]
    
    # Create a unique event_id generator
    unique_id_gen = count(1)  # Start event IDs from 1 and increment

    # Map old event IDs to new unique event IDs
    old_to_new_id = {}
    for event in all_events:
        old_to_new_id[event["event_id"]] = next(unique_id_gen)

    # Update event_id in events
    for event in all_events:
        event["event_id"] = old_to_new_id[event.get("event_id",None)]
        
        # Update dependencies to use new event IDs
        if "event_dependencies" in event and event["event_dependencies"]:
            for dep in event["event_dependencies"]:
                dep["event_id"] = old_to_new_id.get(dep.get("event_id",""), None)  # Replace with new ID

    return all_events


updated_events = assign_unique_event_ids(event_lst)


In [60]:
event_lst[0]

[{'title': 'China’s economic data breakdowns show signs of life amid prevailing doom-and-gloom outlook',
  'event_id': 197,
  'main_event': "China's consumption trend polarised among sectors",
  'who': 'China',
  'what': 'Consumption trend polarised among sectors, with service consumption such as concerts, tourism and catering performing better than the sale of goods',
  'where': 'China',
  'when': 'This year',
  'cause': 'Worsening income prospects suppressing the sale of goods',
  'consequences': 'Service sectors like concerts, tourism and catering have seen better performance',
  'event_dependencies': []},
 {'event_id': 198,
  'main_event': 'Record high booking value on Chinese travel platforms',
  'who': 'Trip.com and Tongcheng Travel',
  'what': 'Record high total booking value',
  'where': 'China',
  'when': 'July',
  'cause': 'High demand during the summer holiday',
  'consequences': 'Increased business for Chinese travel platforms Trip.com and Tongcheng Travel',
  'event_depend

In [58]:
updated_events[0]

{'title': 'China’s economic data breakdowns show signs of life amid prevailing doom-and-gloom outlook',
 'event_id': 197,
 'main_event': "China's consumption trend polarised among sectors",
 'who': 'China',
 'what': 'Consumption trend polarised among sectors, with service consumption such as concerts, tourism and catering performing better than the sale of goods',
 'where': 'China',
 'when': 'This year',
 'cause': 'Worsening income prospects suppressing the sale of goods',
 'consequences': 'Service sectors like concerts, tourism and catering have seen better performance',
 'event_dependencies': []}

In [59]:
updated_events[1]

{'event_id': 198,
 'main_event': 'Record high booking value on Chinese travel platforms',
 'who': 'Trip.com and Tongcheng Travel',
 'what': 'Record high total booking value',
 'where': 'China',
 'when': 'July',
 'cause': 'High demand during the summer holiday',
 'consequences': 'Increased business for Chinese travel platforms Trip.com and Tongcheng Travel',
 'event_dependencies': [{'parent_event_id': 1,
   'relation': 'INFLUENCED',
   'parent_event_details': {'title': 'Singapore shares close flat amid mixed regional trading; STI dips 0.03%',
    'event_id': 197,
    'main_event': 'Singapore stocks slipped into slightly negative territory',
    'who': 'Stock traders in Singapore',
    'what': 'Trading of stocks resulting in a slight dip',
    'where': 'Singapore',
    'when': 'Dec 9',
    'cause': 'Mixed regional trading',
    'consequences': 'The Straits Times Index (STI) inched down by 1.24 points or 0.03 per cent to 3,794.92',
    'event_dependencies': [{'parent_event_id': 2,
      '

In [47]:
from collections import defaultdict

def map_event_dependencies(data):
    # Flatten the list of lists into a single list of events
    all_events = [event for sublist in data for event in sublist]
    
    # Create a dictionary mapping event_id to event details
    event_dict = {event["event_id"]: event for event in all_events}
    
    for event in all_events:
        if "event_dependencies" in event and event["event_dependencies"]:
            mapped_dependencies = [
                {
                    "parent_event": event_dict[dep["event_id"]]["main_event"] if dep["event_id"] in event_dict else None,
                    "relation": dep["relation"]
                }
                for dep in event["event_dependencies"]
            ]
            event["event_dependencies"] = mapped_dependencies
    
    return all_events

mapped_events = map_event_dependencies(event_lst)
for event in mapped_events:
    print(event)


KeyError: 'event_id'

In [12]:
pd.set_option("max.colwidth",0)

## Summarize the information

In [50]:
text = """
"Climate change is causing more frequent and intense heatwaves around the world. Rising global temperatures have led to melting glaciers and rising sea levels, which threaten coastal cities. In 2021, wildfires in California destroyed thousands of homes due to prolonged drought and high temperatures. Scientists warn that without significant reductions in carbon emissions, these extreme weather events will become even more common. Governments have pledged to cut emissions by investing in renewable energy sources like solar and wind power."
"""

prompt_template = PromptTemplate(
    input_variables=["text"],
    template=(
        "Generate an abstractive summary in JSON format from the following text. "
        "Use the following JSON structure:\n\n"
        "{{\n"
        '  "main_event": "[Describe the main event concisely]",\n'
        '  "who": "[Who did it or who was involved]",\n'
        '  "what": "[What happened]",\n'
        '  "where": "[Where did it happen]",\n'
        '  "when": "[When did it happen]",\n'
        '  "cause": "[What caused the event]",\n'
        '  "consequences": "[What were the outcomes or impacts of the event]"\n'
        "}}\n\n"
        "Ignore less informative details and focus on key events only.\n\nText:\n{text}"
    )
)


# Create a ChatOpenAI LLM instance (chat-based model)
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Create a LangChain with the defined prompt template
chain = LLMChain(llm=llm, prompt=prompt_template)

# Run the chain and print the abstractive summary
response = chain.run(text)
print("Abstractive Summary with Independent Sentences:\n")
print(response)



Abstractive Summary with Independent Sentences:

{
  "main_event": "Climate change causing more frequent and intense heatwaves",
  "who": "Global population, governments, scientists",
  "what": "Rising global temperatures have led to melting glaciers and rising sea levels, causing wildfires and threatening coastal cities. Governments are pledging to cut emissions by investing in renewable energy sources.",
  "where": "Worldwide",
  "when": "Ongoing, with a notable event in 2021",
  "cause": "Climate change due to increased carbon emissions",
  "consequences": "Destruction of homes due to wildfires, threat to coastal cities due to rising sea levels, and a call for significant reductions in carbon emissions"
}


In [51]:
import json

In [52]:
data = json.loads(response)

In [54]:
import spacy
import pandas as pd
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

# Replace with your OpenAI API key
import os

# Load spaCy's English model for NER
nlp = spacy.load("en_core_web_sm")

/Users/thetsu/anaconda3/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [53]:
def split_into_triplets(data):
    main_event = data['main_event']
    triplets = []

    # Main Event Relationships
    triplets.append((main_event, "consists_of", data['who']))
    triplets.append((main_event, "happens_on", data['where']))
    triplets.append((main_event, "happens_on", data['when']))

    # Split 'cause' into separate triplets
    causes = data['cause'].split(", ")
    for cause in causes:
        triplets.append((main_event, "caused_by", cause))

    # Split 'consequences' into separate triplets
    consequences = data['consequences'].split(". ")
    for consequence in consequences:
        if consequence:
            triplets.append((main_event, "has_effect", consequence))

    # Split 'what' into separate triplets
    what_parts = data['what'].split(". ")
    for what in what_parts:
        if what:
            triplets.append((main_event, "is", what))

    return triplets

# Extract triplets
triplets = split_into_triplets(data)
print("Extracted Triplets:")
for triplet in triplets:
    print(triplet)


Extracted Triplets:
('Climate change causing more frequent and intense heatwaves', 'consists_of', 'Global population, governments, scientists')
('Climate change causing more frequent and intense heatwaves', 'happens_on', 'Worldwide')
('Climate change causing more frequent and intense heatwaves', 'happens_on', 'Ongoing, with a notable event in 2021')
('Climate change causing more frequent and intense heatwaves', 'caused_by', 'Climate change due to increased carbon emissions')
('Climate change causing more frequent and intense heatwaves', 'has_effect', 'Destruction of homes due to wildfires, threat to coastal cities due to rising sea levels, and a call for significant reductions in carbon emissions')
('Climate change causing more frequent and intense heatwaves', 'is', 'Rising global temperatures have led to melting glaciers and rising sea levels, causing wildfires and threatening coastal cities')
('Climate change causing more frequent and intense heatwaves', 'is', 'Governments are pledgi

## Expand the triplets

In [75]:
### The main purpose is to expland the data in "consists_of, happends_on",
### 
expanded_triplets = []
for data in triplets:
    sub,rel, obj = data
    if rel == "consists_of" and (",") in obj:
        for obj_split in obj.split(","): 
            expanded_triplets.append((sub, rel, obj_split))
    if rel == "happens_on" and (",") in obj:
        for obj_split in obj.split(","): 
            expanded_triplets.append((sub, rel, obj_split))
    else:
        expanded_triplets.append((sub, rel, obj))

In [76]:
expanded_triplets

[('Climate change causing more frequent and intense heatwaves',
  'consists_of',
  'Global population'),
 ('Climate change causing more frequent and intense heatwaves',
  'consists_of',
  ' governments'),
 ('Climate change causing more frequent and intense heatwaves',
  'consists_of',
  ' scientists'),
 ('Climate change causing more frequent and intense heatwaves',
  'consists_of',
  'Global population, governments, scientists'),
 ('Climate change causing more frequent and intense heatwaves',
  'happens_on',
  'Worldwide'),
 ('Climate change causing more frequent and intense heatwaves',
  'happens_on',
  'Ongoing'),
 ('Climate change causing more frequent and intense heatwaves',
  'happens_on',
  ' with a notable event in 2021'),
 ('Climate change causing more frequent and intense heatwaves',
  'caused_by',
  'Climate change due to increased carbon emissions'),
 ('Climate change causing more frequent and intense heatwaves',
  'has_effect',
  'Destruction of homes due to wildfires, thre

## Extract the entities for what (is), caused_by, and has_effect

In [69]:
## Named Entity Relations Extraction


for triplet in expanded_triplets:
    # Define the text to analyze
    text = 

    # Extract entities using spaCy
    def extract_entities(text):
        doc = nlp(text)
        entities = {ent.text: ent.label_ for ent in doc.ents}
        return entities

    entities = extract_entities(text)
    print("Extracted Entities:")
    for entity, label in entities.items():
        print(f"{entity}: {label}")


Extracted Entities:
Myanmar: GPE
2022: DATE


In [91]:
prompt_template = PromptTemplate(
    input_variables=["text"],
    template=(
        "Extract simplified and normalized triplets (subject, relation, object) from the following text. "
        "Use short and meaningful keywords for each part, avoiding long phrases or redundancy. "
        "Lemmatize terms and ensure each triplet has exactly three parts: "
        "a single subject, a relation, and a single object. Return the result as a clean list of triplets.\n\nText:\n{text}"
    )
)

# Create a ChatOpenAI LLM instance
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Create a LangChain with the defined prompt template
chain = LLMChain(llm=llm, prompt=prompt_template)

# Function to clean and fix triplets
def clean_and_fix_triplets(response):
    # Extract triplets using regex
    triplet_pattern = re.compile(r"\(([^)]+)\)")
    triplets = triplet_pattern.findall(response)
    
    cleaned_triplets = []
    for triplet in triplets:
        # Remove extra quotes and split by commas
        parts = [part.strip().strip('"') for part in triplet.split(",")]
        
        # Fix triplets with more than 3 parts
        if len(parts) > 3:
            subject = " and ".join(parts[:-2])
            relation = parts[-2]
            obj = parts[-1]
            cleaned_triplets.append((subject, relation, obj))
        elif len(parts) == 3:
            cleaned_triplets.append(tuple(parts))
    
    return cleaned_triplets


def annotate_triplets_with_entities(triplets, entities):
    enriched_triplets = []
    for triplet in triplets:
        subject, relation, obj = triplet
        subject_type = entities.get(subject, "UNKNOWN")
        object_type = entities.get(obj, "UNKNOWN")
        enriched_triplets.append((subject, subject_type, relation, obj, object_type))
    return enriched_triplets


### Expand all
final_triplets = []
for triplets in expanded_triplets:
    ## Get the what, is_caused_by and has_effect
    main_event, main_rel, main_obj = triplets
    if main_rel in (["is", "has_effect", "caused_by"]):
        target_obj_sent = main_obj
        # Run the chain to extract triplets
        response = chain.run(target_obj_sent)


        # Clean and fix extracted triplets
        fixed_triplets = clean_and_fix_triplets(response)

        # Annotate triplets with entity types
#         enriched_triplets = annotate_triplets_with_entities(fixed_triplets, entities)
        final_triplets.append((main_event,main_rel,(fixed_triplets)))
        
    else:
        final_triplets.append(triplets)
        
        

In [88]:
final_triplets

[('Climate change causing more frequent and intense heatwaves',
  'consists_of',
  'Global population'),
 ('Climate change causing more frequent and intense heatwaves',
  'consists_of',
  ' governments'),
 ('Climate change causing more frequent and intense heatwaves',
  'consists_of',
  ' scientists'),
 ('Climate change causing more frequent and intense heatwaves',
  'consists_of',
  'Global population, governments, scientists'),
 ('Climate change causing more frequent and intense heatwaves',
  'happens_on',
  'Worldwide'),
 ('Climate change causing more frequent and intense heatwaves',
  'happens_on',
  'Ongoing'),
 ('Climate change causing more frequent and intense heatwaves',
  'happens_on',
  ' with a notable event in 2021'),
 ('Climate change causing more frequent and intense heatwaves',
  'caused_by',
  [('Climate change',
    'UNKNOWN',
    'due to',
    'increased carbon emissions',
    'UNKNOWN')]),
 ('Climate change causing more frequent and intense heatwaves',
  'has_effect'

In [93]:
final_triplets

[('Climate change causing more frequent and intense heatwaves',
  'consists_of',
  'Global population'),
 ('Climate change causing more frequent and intense heatwaves',
  'consists_of',
  ' governments'),
 ('Climate change causing more frequent and intense heatwaves',
  'consists_of',
  ' scientists'),
 ('Climate change causing more frequent and intense heatwaves',
  'consists_of',
  'Global population, governments, scientists'),
 ('Climate change causing more frequent and intense heatwaves',
  'happens_on',
  'Worldwide'),
 ('Climate change causing more frequent and intense heatwaves',
  'happens_on',
  'Ongoing'),
 ('Climate change causing more frequent and intense heatwaves',
  'happens_on',
  ' with a notable event in 2021'),
 ('Climate change causing more frequent and intense heatwaves',
  'caused_by',
  [('Climate change', 'due to', 'increased carbon emissions')]),
 ('Climate change causing more frequent and intense heatwaves',
  'has_effect',
  [('homes', 'destruction', 'wildfir

## Build Knowledge Graph

In [95]:
from neo4j import GraphDatabase

# # Create a driver with encrypted connection for Neo4j Aura
# driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# def load_triples(triples):
#     try:
#         with driver.session() as session:
#             for triple in triples:
#                 session.write_transaction(create_relationship, triple)
#     except Exception as e:
#         print(f"Error: {e}")
#     finally:
#         driver.close()

# def create_relationship(tx, triple):
#     # Use dynamic relationship type
#     query = f"""
#     MERGE (a:Event {{name: $a}})
#     MERGE (b:Event {{name: $b}})
#     MERGE (a)-[:{triple[1]}]->(b)
#     """
#     params = {"a": triple[0], "b": triple[2]}
#     tx.run(query, params)


# load_triples(triples)

In [98]:
# Flatten nested triplets with direct and nested relationships
def flatten_triples(triples):
    flat_triples = []
    for triple in triples:
        if isinstance(triple[2], list):  # Check if the object is a nested list of triplets
            for nested_triple in triple[2]:
                # 1. Extract direct relationships
                flat_triples.append((triple[0], triple[1], nested_triple[0]))
                
                # 2. Extract nested relationships
                flat_triples.append((nested_triple[0], nested_triple[1], nested_triple[2]))
        else:
            flat_triples.append(triple)
    return flat_triples

# Flatten the nested triplets
flat_triples = flatten_triples(final_triplets)

# Print flattened triplets for verification
print("Flattened Triples:")
for t in flat_triples:
    print(t)

Flattened Triples:
('Climate change causing more frequent and intense heatwaves', 'consists_of', 'Global population')
('Climate change causing more frequent and intense heatwaves', 'consists_of', ' governments')
('Climate change causing more frequent and intense heatwaves', 'consists_of', ' scientists')
('Climate change causing more frequent and intense heatwaves', 'consists_of', 'Global population, governments, scientists')
('Climate change causing more frequent and intense heatwaves', 'happens_on', 'Worldwide')
('Climate change causing more frequent and intense heatwaves', 'happens_on', 'Ongoing')
('Climate change causing more frequent and intense heatwaves', 'happens_on', ' with a notable event in 2021')
('Climate change causing more frequent and intense heatwaves', 'caused_by', 'Climate change')
('Climate change', 'due to', 'increased carbon emissions')
('Climate change causing more frequent and intense heatwaves', 'has_effect', 'homes')
('homes', 'destruction', 'wildfires')
('Clim

In [100]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Neo4j insertion logic
def load_triples(triples):
    try:
        with driver.session() as session:
            for triple in triples:
                session.write_transaction(create_relationship, triple)
    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.close()

def create_relationship(tx, triple):
    # Use dynamic relationship type
    query = f"""
    MERGE (a:Entity {{name: $a}})
    MERGE (b:Entity {{name: $b}})
    MERGE (a)-[:{triple[1].replace(" ", "_").upper()}]->(b)
    """
    params = {"a": triple[0], "b": triple[2]}
    tx.run(query, params)

# Load flattened triplets into Neo4j
load_triples(flat_triples)

/var/folders/xn/dm_gjx4d16363nrz6_260mgc0000gn/T/ipykernel_34505/2370714826.py:8: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, triple)


## DONE HERE

In [89]:
df_triplets = pd.DataFrame(final_triplets, columns=["Main_Event", "Relation", "Object"])

In [90]:
df_triplets

,Main_Event,Relation,Object
0,Climate change causing more frequent and intense heatwaves,consists_of,Global population
1,Climate change causing more frequent and intense heatwaves,consists_of,governments
2,Climate change causing more frequent and intense heatwaves,consists_of,scientists
3,Climate change causing more frequent and intense heatwaves,consists_of,"Global population, governments, scientists"
4,Climate change causing more frequent and intense heatwaves,happens_on,Worldwide
5,Climate change causing more frequent and intense heatwaves,happens_on,Ongoing
6,Climate change causing more frequent and intense heatwaves,happens_on,with a notable event in 2021
7,Climate change causing more frequent and intense heatwaves,caused_by,"[(Climate change, UNKNOWN, due to, increased carbon emissions, UNKNOWN)]"
8,Climate change causing more frequent and intense heatwaves,has_effect,"[(homes, UNKNOWN, destruction, wildfires, UNKNOWN), (coastal cities, UNKNOWN, threat, rising sea levels, UNKNOWN), (call, UNKNOWN, demand, reductions in carbon emissions, UNKNOWN)]"
9,Climate change causing more frequent and intense heatwaves,is,"[(global temperature, UNKNOWN, rise, melting glaciers, UNKNOWN), (global temperature, UNKNOWN, rise, rising sea levels, UNKNOWN), (rising sea levels, UNKNOWN, cause, wildfires, UNKNOWN), (rising sea levels, UNKNOWN, threaten, coastal cities, UNKNOWN)]"


In [92]:
df_triplets = pd.DataFrame(final_triplets, columns=["Main_Event", "Relation", "Object"])
df_triplets

,Main_Event,Relation,Object
0,Climate change causing more frequent and intense heatwaves,consists_of,Global population
1,Climate change causing more frequent and intense heatwaves,consists_of,governments
2,Climate change causing more frequent and intense heatwaves,consists_of,scientists
3,Climate change causing more frequent and intense heatwaves,consists_of,"Global population, governments, scientists"
4,Climate change causing more frequent and intense heatwaves,happens_on,Worldwide
5,Climate change causing more frequent and intense heatwaves,happens_on,Ongoing
6,Climate change causing more frequent and intense heatwaves,happens_on,with a notable event in 2021
7,Climate change causing more frequent and intense heatwaves,caused_by,"[(Climate change, due to, increased carbon emissions)]"
8,Climate change causing more frequent and intense heatwaves,has_effect,"[(homes, destruction, wildfires), (coastal cities, threat, rising sea levels), (call, reduction, carbon emissions)]"
9,Climate change causing more frequent and intense heatwaves,is,"[(global temperature, rise, melting glaciers), (melting glaciers, cause, rising sea levels), (rising sea levels, cause, wildfires), (rising sea levels, threaten, coastal cities)]"


In [22]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
text = "China sanctions two US defence firms for selling Taiwan weapons"
labels = ["Economy", "Technology", "Politics", "Crime", "Science", "Health"]
result = classifier(text, candidate_labels=labels)
print(result)


Device set to use cpu


{'sequence': 'China sanctions two US defence firms for selling Taiwan weapons', 'labels': ['Economy', 'Crime', 'Politics', 'Technology', 'Science', 'Health'], 'scores': [0.26702791452407837, 0.24168507754802704, 0.21354123950004578, 0.14741374552249908, 0.06677037477493286, 0.06356166303157806]}


In [24]:
def predict(text):
    result = classifier(text, candidate_labels=labels)
    highest_score_index = result['scores'].index(max(result['scores']))
    highest_score_label = result['labels'][highest_score_index]

    return(highest_score_label)

In [25]:
subset_df = df[["Published Date","Title"]]

In [26]:
subset_df["topic"] = subset_df["Title"].apply(lambda x:predict(x))

/tmp/ipykernel_251060/1729149219.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["topic"] = subset_df["Title"].apply(lambda x:predict(x))


In [27]:
subset_df.head()

,Published Date,Title,topic
6458,2023-09-15T02:27:48Z,China’s economic data breakdowns show signs of life amid prevailing doom-and-gloom outlook,Economy
3333,2024-08-06T07:30:00Z,5 years’ jail for ‘key cog’ in cryptocurrency investment scam involving more than $1.8m,Crime
5729,2023-11-17T07:18:00Z,"Australia, India at risk of becoming dumping grounds for polluting cars, report says",Economy
5963,2023-11-01T04:45:00Z,China's new home prices rise for second month in Oct: survey,Economy
1731,2024-11-16T00:24:00Z,"China to retaliate if Trump boosts tariff, says ex-PBOC official",Politics


In [28]:
subset_df["topic"].value_counts()

Economy       24
Politics      12
Crime         5 
Technology    4 
Health        3 
Science       2 
Name: topic, dtype: int64

In [35]:
subset_df[subset_df.topic == "Crime"]

,Published Date,Title,topic
3333,2024-08-06T07:30:00Z,5 years’ jail for ‘key cog’ in cryptocurrency investment scam involving more than $1.8m,Crime
1755,2024-11-15T02:14:00Z,UK government discloses crimes allegedly committed by foreign diplomats,Crime
6486,2023-09-13T13:33:06Z,"Malaysia not 'some banana republic', will go ""very far"" to recover stolen 1MDB funds: PM Anwar",Crime
3127,2024-08-26T05:55:17Z,Jail for mastermind of crypto Ponzi scheme after investors lose S$1.1 million,Crime
4525,2024-05-03T09:23:00Z,"Philippine ship damage, injury to crew 'irresponsible behaviour', says US Defense Secretary Austin",Crime


In [36]:
subset_df["content"] = df["Content"]

/tmp/ipykernel_251060/2375297318.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["content"] = df["Content"]


In [40]:
print(subset_df["content"].iloc[0])

Apart from a divergence among different income groups, China’s consumption trend this year is polarised among sectors, as service consumption such as concerts, tourism and catering have been better than the sale of goods – which has been more suppressed by worsening income prospects, Zhang said.

Sandalwood data showed that the total booking value on two major Chinese travel platforms, Trip.com and Tongcheng Travel, hit a record high in July, thanks to the high demand during the summer holiday.

Despite the slightly weakening momentum in August, it still maintained double-digit growth: in the first half of August, bookings on Trip.com increased by 85 per cent by value, year on year, and on Tongcheng Travel they increased by 60 per cent.

In some major cities, the subway passenger volume has also been well above pre-pandemic levels in the past few months, though that of the top three metropolises of Beijing, Shanghai and Guangzhou have yet to fully recover to 2019’s level, according to 

In [41]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import json

In [47]:
# Set up your OpenAI API key

# Initialize the LLM (GPT-4 or GPT-3.5)
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-3.5-turbo",  # You can use "gpt-3.5-turbo" for faster results
    temperature=0.3  # Lower temp for more factual responses
)

def extract_information(article):
    """
    Extracts key information from the news article focusing on events and returns it in JSON format.
    """
    system_message = SystemMessage(
        content="You are an expert in news analysis. Extract and summarize key information from the given article, focusing on the main events. Respond only in JSON format."
    )
    
    user_message = HumanMessage(
        content=f"""
        Extract the key information from the following news article. Focus on identifying the **main event**, **date**, **location**, **involved entities** (such as organizations, countries, or individuals), **causes**, **consequences**, and any significant **quotes or statements**.

        Return the information as a JSON object with the following fields:
        {{
            "main_event": "Brief description of the main event",
            "date": "Date if available",
            "location": "Location if available",
            "involved_entities": ["List", "of", "entities"],
            "cause": "Reason behind the event",
            "consequence": "Outcome or impact",
            "key_statement": "Quoted statement if relevant"
        }}

        News Article:
        {article}
        """
    )

    response = llm([system_message, user_message])
    return response.content

total_articles  = []

for news_article in subset_df["content"]:
    summary_json = json.loads(extract_information(news_article))
    total_articles.append(summary_json)

In [48]:
total_articles[:10]

[{'main_event': "Polarisation of China's consumption trend among sectors",
  'date': 'July to August',
  'location': 'China',
  'involved_entities': ['Sandalwood data',
   'Trip.com',
   'Tongcheng Travel',
   'Wind (financial data provider)',
   'Dengta (movie industry data provider)',
   'Nomura',
   'Hang Seng Bank China',
   'Capital Economics'],
  'cause': 'Worsening income prospects affecting the sale of goods, pent-up need for tourism and entertainment post-COVID-19 lockdowns',
  'consequence': 'Record high summer box office, surge in tourism and entertainment spending, concerns about sustainability of consumer spending',
  'key_statement': 'Ultimately, those non-essential consumptions are constrained by household income, which is determined by overall economic growth. If you are looking at any place other than housing, you are wasting your time.'},
 {'main_event': 'A&A Blockchain Innovation running a Ponzi scheme under the guise of a cryptocurrency investment scheme',
  'date':

In [60]:
# Create triples
triples = []
for item in total_articles:
    event = item["main_event"]
    if item["involved_entities"]:
        for entity in item["involved_entities"]:
            triples.append((event, "INVOLVES", entity))
    if item["cause"]:
        triples.append((event, "CAUSES", item["cause"]))
    if item["consequence"]:
        triples.append((event, "LEADS_TO", item["consequence"]))
    if item["location"]:
        for loc in item["location"]:
            triples.append((event, "HAPPENED_AT", loc))

In [61]:
triples[:10]

[("Polarisation of China's consumption trend among sectors",
  'INVOLVES',
  'Sandalwood data'),
 ("Polarisation of China's consumption trend among sectors",
  'INVOLVES',
  'Trip.com'),
 ("Polarisation of China's consumption trend among sectors",
  'INVOLVES',
  'Tongcheng Travel'),
 ("Polarisation of China's consumption trend among sectors",
  'INVOLVES',
  'Wind (financial data provider)'),
 ("Polarisation of China's consumption trend among sectors",
  'INVOLVES',
  'Dengta (movie industry data provider)'),
 ("Polarisation of China's consumption trend among sectors",
  'INVOLVES',
  'Nomura'),
 ("Polarisation of China's consumption trend among sectors",
  'INVOLVES',
  'Hang Seng Bank China'),
 ("Polarisation of China's consumption trend among sectors",
  'INVOLVES',
  'Capital Economics'),
 ("Polarisation of China's consumption trend among sectors",
  'CAUSES',
  'Worsening income prospects affecting the sale of goods, pent-up need for tourism and entertainment post-COVID-19 lockdo

In [62]:
from neo4j import GraphDatabase

In [65]:
from neo4j import GraphDatabase

# Create a driver with encrypted connection for Neo4j Aura
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def load_triples(triples):
    try:
        with driver.session() as session:
            for triple in triples:
                session.write_transaction(create_relationship, triple)
    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.close()

def create_relationship(tx, triple):
    # Use dynamic relationship type
    query = f"""
    MERGE (a:Event {{name: $a}})
    MERGE (b:Event {{name: $b}})
    MERGE (a)-[:{triple[1]}]->(b)
    """
    params = {"a": triple[0], "b": triple[2]}
    tx.run(query, params)


load_triples(triples)

/tmp/ipykernel_251060/2954216185.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, triple)
